In [1]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import ng_models
import matplotlib.pyplot as plt
from collections import Counter
import stanza

#stanza.download('pl')

%matplotlib notebook
importlib.reload(funcs)
importlib.reload(ng_models)

stopwords = open('00_dane/polish.stopwords.txt','r').read().split()
stopwords.append(['.', ',','!', '?'])

In [2]:
s = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

In [3]:
sp =pd.read_csv('00_dane/rawest.csv', sep='|', encoding='UTF-8')

In [16]:
sp = sp.drop(columns = 'Unnamed: 0', axis=1)
#sp.columns

In [4]:
sp.iloc[66666]['speech_rawest']

'2500 szkół , Bóg . . . spojrzeć wstecz , zauważyć , wiek ziemia polski wielki skala , , szkół zamykać . raz ostatni katastrofa oświatowy dotknąć Polak wiek XIX następstwo utrata niepodległość . drugi połowa wiek XIX wiek XX rok mozolny odbud polski system oświatowy . 200 rok tworzyć ów system , budować otwierać szkoła . system niszczyć ? pytanie skierować minister , poseł koalicja rządzić . dziać ? dawien dawny rządzić odczuwać pokusa redukcja wydatek oświata . często urzędnik niemiecki rosyjski , nazistowski sowiecki . urzędnik polski uczestniczyć podobny proceder ? szkoła likwidować samorząd ? opinia . likwidować jedynie ręka . rząd , podobnie przypadek szpital przedszkola , uczynić samorząd jedynie likwidator . podkreślać likwidator . pojedynczy szkół ? . opinia , likwidywać tysiąc szkół , de facty likwidacja system oświata . rząd czuć odpowiedzialny likwidować szkoła ? los środowisko pozbawić kulturotwórczy rola szkoła ? prawo sprawiedliwość , wnosić projekt ustawa zmiana ustawa s

In [3]:
sp.tail(10)

,date_,speech_rawer,full_name,party_final,speech_rawest
158872,31-08-2011,Panie Przewodniczący! Z należnym szacunkiem p...,Jan Kulas,Platforma Obywatelska,NaN
158873,31-08-2011,Moje oświadczenie poselskie poświęcam działaln...,Jan Kulas,Platforma Obywatelska,NaN
158874,31-08-2011,"Wyobraźmy sobie, że gdzieś w tzw. cywilizowan...",Paweł Graś,Platforma Obywatelska,NaN
158875,31-08-2011,"Doszło do skandalicznego głosowania, albowiem...",Paweł Graś,Platforma Obywatelska,NaN
158876,31-08-2011,Z głębokim smutkiem i wielkim zażenowaniem pr...,Paweł Graś,Platforma Obywatelska,NaN
158877,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,NaN
158878,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,NaN
158879,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,NaN
158880,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,NaN
158881,31-08-2011,Szanowna Pani Minister! Bardzo ważnym zadanie...,Tadeusz Tomaszewski,Sojusz Lewicy Demokratycznej,NaN


In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined2.csv',  sep='|', encoding='UTF-8')

## Przygotowanie zbioru danych pod analizę tematyczną (i poźniej pod analizę sentymentu również):
- lematyzacja
- usunięcie stop words
- wektoryzacja

In [72]:
sp_joined.iloc[6]['speech_raw']

' Andrzej Tadeusz Mazurkiewicz, Konfederacja Polski Niepodległej. Panie Marszałku! Wysoka Izbo! Proszę państwa, na tej sali padają jakieś dziwne głosy. Naprawdę zgadzam się ze zdaniem mojego kolegi Mirka Lewandowskiego, aby mówić językiem prostym. Padają argumenty, iż osąd co do stanu wojennego należy pozostawić historii. Tak, pozostawmy to historii, bo historycy mogą się tym zająć. Natomiast w tej chwili mamy rzeczywistość, którą musimy ocenić, i ocenimy ją tak, jak odczuwaliśmy to wcześniej. Padają tu opinie, iż różni eksperci wypowiadali się na ten temat i są różne ekspertyzy. Nie wiem z jakiego języka były one tłumaczone, ale rzeczywiście jest ich dużo. Proszę państwa, pan mecenas Taylor w imieniu Klubu Parlamentarnego Unia Demokratyczna powiedział, że poczucie sprawiedliwości trzeba poświęcić w imię poczucia bezpieczeństwa. Otóż nie, właśnie poczucie bezpieczeństwa wymaga sprawiedliwości, wymaga zwykłej ludzkiej sprawiedliwości. I te wnioski my dzisiaj na tej sali musimy wyciągnąć

In [12]:
sp = sp_joined[['date_', 'speech_rawer', 'full_name', 'party_final']]

In [8]:
sp.tail(10)

,Unnamed: 0,date_,speech_rawer,full_name,party_final,speech_rawest
158875,158875,31-08-2011,Panie Przewodniczący! Z należnym szacunkiem p...,Jan Kulas,Platforma Obywatelska,NaN
158876,158876,31-08-2011,Moje oświadczenie poselskie poświęcam działaln...,Jan Kulas,Platforma Obywatelska,NaN
158877,158877,31-08-2011,"Wyobraźmy sobie, że gdzieś w tzw. cywilizowan...",Paweł Graś,Platforma Obywatelska,NaN
158878,158878,31-08-2011,"Doszło do skandalicznego głosowania, albowiem...",Paweł Graś,Platforma Obywatelska,NaN
158879,158879,31-08-2011,Z głębokim smutkiem i wielkim zażenowaniem pr...,Paweł Graś,Platforma Obywatelska,NaN
158880,158880,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,NaN
158881,158881,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,NaN
158882,158882,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,NaN
158883,158883,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,NaN
158884,158884,31-08-2011,Szanowna Pani Minister! Bardzo ważnym zadanie...,Tadeusz Tomaszewski,Sojusz Lewicy Demokratycznej,NaN


### Usunięcie stop words oraz lematyzacja

In [2]:
nlp = stanza.Pipeline('pl')

2020-05-23 17:03:31 INFO: Loading these models for language: pl (Polish):
| Processor | Package |
-----------------------
| tokenize  | lfg     |
| pos       | lfg     |
| lemma     | lfg     |
| depparse  | lfg     |

2020-05-23 17:03:31 INFO: Use device: cpu
2020-05-23 17:03:31 INFO: Loading: tokenize
2020-05-23 17:03:31 INFO: Loading: pos
2020-05-23 17:03:33 INFO: Loading: lemma
2020-05-23 17:03:33 INFO: Loading: depparse
2020-05-23 17:03:35 INFO: Done loading processors!


In [5]:
%%time
x=0
for i in sp[sp['speech_rawest'].isnull()].index:
    
    doc = nlp(sp.loc[i, 'speech_rawer'])
    sp.loc[i, 'speech_rawest']= ' '.join([word.lemma for sent in doc.sentences for word in sent.words if word.lemma not in stopwords])
    if round(100*i/len(sp['speech_rawer']),0)>x:
        x = round(100*i/len(sp['speech_rawer']),0)
        print('Done: ',x,'%')
        sp.to_csv('00_dane/rawest.csv', sep='|', encoding='UTF-8')

Done:  88.0 %
Done:  89.0 %
Done:  90.0 %
Done:  91.0 %
Done:  92.0 %
Done:  93.0 %
Done:  94.0 %
Done:  95.0 %
Done:  96.0 %
Done:  97.0 %
Done:  98.0 %
Done:  99.0 %
Done:  100.0 %
Wall time: 12h 47min 26s


In [6]:
sp.to_csv('00_dane/rawest.csv', sep='|', encoding='UTF-8', index=False)


In [5]:
sp[sp['speech_rawest'].isnull()].index

Int64Index([116830, 116831, 116832, 116833, 116834, 116835, 116836, 116837,
            116838, 116839,
            ...
            158872, 158873, 158874, 158875, 158876, 158877, 158878, 158879,
            158880, 158881],
           dtype='int64', length=42052)

In [39]:
sp[sp['speech_rawer']==' '].index

Int64Index([], dtype='int64')

In [3]:
nlp("sportowy")

[
  [
    {
      "id": "1",
      "text": "sportowy",
      "lemma": "sportowy",
      "upos": "ADJ",
      "xpos": "adj:sg:nom:m3:pos",
      "feats": "Case=Nom|Degree=Pos|Gender=Masc|Number=Sing|SubGender=Masc3",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=0|end_char=8"
    }
  ]
]

In [38]:
sp = sp.drop(index=80002)